**Dealing with a large dataset on your computer** is a common challenge for data analysts. If you need to process or filter the data you can set the `chunksize=` argument of pandas `read_csv()` method to loop through and work with manageable chunks of the data.

If for example, you wanted to work with a large data file (HCPS_data.csv) to pull just the rows where the `HCPS Code` is **99213**, you could read that file to a chunk of 100,000 rows at a time, filter each chunk to the rows with the specified code, save the filtered results of each chunk to a list and concatenate them all together at the end. The syntax would look something like this:

```
code_99213_rows =[]

for chunk in pd.read_csv('HCPS_data.csv', chunksize = 100000):
    code_99213_rows.append(chunk[chunk['HCPS Code'] == '99213']) 
               
code_99213_df = pd.concat(code_99213_rows, ignore_index=True)
```
======================================================================   

To shrink the size of a file so that it loads more quickly, converting a text file (CSV) to binary might make sense. In python, you can work with data to minimize its footprint and then store the resulting object (dataframe) as a [pickle](https://docs.python.org/3/library/pickle.html) file.

In [1]:
import pandas as pd
import pickle

In [2]:
%%time
july = pd.read_csv('../data/july.csv')
july.head()

Wall time: 34.9 s


,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-07-01 00:00:33.550000,36.156678,-86.809004,Powered635135,Powered,22.0,scooter,0.15,Lyft
1,2019-07-01 00:00:34.973000,36.145674,-86.794138,Powered790946,Powered,33.0,scooter,0.15,Lyft
2,2019-07-01 00:00:41.183000,36.179319,-86.751538,Powered570380,Powered,76.0,scooter,0.15,Lyft
3,2019-07-01 00:00:41.620000,36.152111,-86.803821,Powered240631,Powered,43.0,scooter,0.15,Lyft
4,2019-07-01 00:00:45.087000,36.149355,-86.797550,Powered970404,Powered,52.0,scooter,0.15,Lyft


### Now try to reduce the size of the file
- objects require the most space


In [3]:
# june.info()

#### convert the company name to an integer 
- find the unique company names
- assign each company an integer (you can use a dictionary for this step)
- update the `companyname` column to store the integer id for each company

In [4]:
july.companyname.unique()

array(['Lyft', 'Bird', 'Spin', 'Bolt', 'Jump', 'Lime', 'Gotcha'],
      dtype=object)

In [5]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}

In [6]:
july.companyname = july.companyname.replace(company_dict)

#### next convert `pubdatetime` to a datetime 

In [7]:
july.pubdatetime = pd.to_datetime(july.pubdatetime)
july.head(2)

,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-07-01 00:00:33.550,36.156678,-86.809004,Powered635135,Powered,22.0,scooter,0.15,1
1,2019-07-01 00:00:34.973,36.145674,-86.794138,Powered790946,Powered,33.0,scooter,0.15,1


#### Next remove unneeded data
#### keep just the scooters

In [8]:
july.sumdgroup.unique()

array(['scooter', 'Scooter', 'bicycle'], dtype=object)

In [9]:
july_scooters = july.loc[july.sumdgroup.isin(['scooter', 'Scooter'])]

#### keep just the columns we want to work with

In [10]:
july_scooters = july_scooters[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

#### check `.info()` again

In [11]:
july_scooters.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25066524 entries, 0 to 25075444
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   chargelevel  float64       
 5   companyname  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 1.3+ GB


#### The only object datatype remaining is sumdid (an alphanumeric unique identifier)
- time to pickle

In [12]:
july_scooters.to_pickle("../data/july.pkl")

In [13]:
%%time
july_test = pd.read_pickle("../data/july.pkl")

Wall time: 1.92 s
